In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-01"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1044,2024-10-01,Europa Eurocopa,13:00,Bahcesehir Kol.,Besiktas,2.32,1.56,163.5,1.85,1.85,2.5,1.95,1.72,Iy1HK6XR,0.431034,0.641026,0.540541,0.540541,0.072060,0.0,0.6,NaN,NaN,NaN,NaN,NaN,141.906,12.033507,0.084799,0.000,149.968,0.00,158.40,0.0,0.0,0.0,0.0,0.277011,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,-2.22,-0.444,-1.261261,0.000000,0.0,0.000000
1045,2024-10-01,Europa Eurocopa,13:00,Lietkabelis,Hapoel Jerusalem,2.53,1.48,160.5,1.88,1.82,2.5,2.05,1.59,xtYsySH8,0.395257,0.675676,0.531915,0.549451,0.070933,0.8,0.0,NaN,NaN,247.974,82.714921,0.333563,118.022,21.110806,0.178872,192.710,0.000,273.24,131.67,1.0,0.0,0.0,0.0,0.370305,0.022933,0.178719,1.46,0.292,5.239726,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1046,2024-10-01,Europa Eurocopa,14:00,Ulm,Joventut,2.45,1.51,164.5,1.86,1.84,2.5,1.99,1.63,AZudDOP1,0.408163,0.662252,0.537634,0.543478,0.070415,0.2,0.4,NaN,NaN,160.492,50.899351,0.317146,190.540,76.161331,0.399713,140.804,183.744,165.90,147.84,0.0,0.0,0.0,0.0,0.335697,0.007644,0.140640,-2.81,-0.562,-2.580071,0.000000,0.0,0.000000,-0.09,-0.018,-28.333333,0.000000,0.0,0.000000
1047,2024-10-01,Europa Eurocopa,14:45,Hapoel Tel-Aviv,BC Wolves,1.05,8.55,166.5,1.88,1.82,-17.5,2.01,1.68,lKimFpek,0.952381,0.116959,0.531915,0.549451,0.069340,0.8,0.0,NaN,NaN,177.818,55.517021,0.312213,228.202,106.797938,0.467997,142.438,302.684,143.85,362.95,0.0,0.0,0.0,0.0,1.104854,0.022933,0.126474,-2.75,-0.550,-0.090909,0.000000,0.0,0.000000,2.47,0.494,15.283401,0.000000,0.0,0.000000
1048,2024-10-01,Europa Eurocopa,15:00,Cedevita Olimpija,Venezia,2.15,1.65,166.5,1.83,1.87,1.5,1.95,1.72,lx3NGcWg,0.465116,0.606061,0.546448,0.534759,0.071177,0.4,0.8,NaN,NaN,149.490,60.427759,0.404226,137.240,48.618194,0.354257,154.996,217.802,153.72,127.80,0.0,0.0,0.0,0.0,0.186081,0.015289,0.088629,-2.29,-0.458,-2.510917,0.559137,0.5,-0.059137,-2.04,-0.408,-1.593137,0.000000,0.0,0.000000
1049,2024-10-01,Eua Wnba,20:30,New York Liberty F,Las Vegas Aces F,1.61,2.33,163.5,1.86,1.94,-4.5,2.03,1.73,YR9N6Nig,0.621118,0.429185,0.537634,0.515464,0.050303,0.8,0.0,NaN,NaN,125.762,29.529740,0.234807,140.200,37.918810,0.270462,115.856,0.000,134.85,190.19,0.0,0.0,0.0,0.0,0.258435,0.029773,0.112836,0.27,0.054,11.296296,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1050,2024-10-01,Eua Wnba,22:30,Minnesota Lynx F,Connecticut Sun F,1.51,2.58,151.5,1.86,1.92,-5.5,2.01,1.78,jy4aYbMa,0.662252,0.387597,0.537634,0.520833,0.049849,0.4,0.4,NaN,NaN,113.130,14.209842,0.125606,139.504,31.082428,0.222807,121.192,147.730,108.50,180.31,0.0,0.0,0.0,0.0,0.369978,0.022448,0.085823,0.41,0.082,6.219512,0.000000,0.0,0.000000,-0.11,-0.022,-71.818182,0.645379,0.8,0.154621
1051,2024-10-01,Europa Liga Dos Campeões,13:00,AEK Athens,Maccabi Ironi Ramat Gan,1.37,2.99,162.5,1.80,1.94,-7.5,1.97,1.72,lzE7okYH,0.729927,0.334448,0.555556,0.515464,0.064375,0.0,0.0,NaN,NaN,166.244,37.893671,0.227940,NaN,NaN,NaN,0.000,0.000,183.28,0.00,0.0,0.0,0.0,0.0,0.525465,0.052938,0.095814,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1052,2024-10-01,Europa Liga Dos Campeões,13:00,VEF Riga,Bonn,2.41,1.54,160.5,1.87,1.87,2.5,2.01,1.69,23o5nBKa,0.414938,0.649351,0.534759,0.534759,0.064288,0.0,0.4,NaN,NaN,265.442,167.110955,0.629557,164.610,37.553586,0.228137,0.000,177.624,255.64,208.25,1.0,0.0,0.0,0.0,0.311485,0.000000,0.122310,0.00,0.000,inf,0.000000,0.0,0.000000,1.55,0.310,1.741935,0.000000

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
